In [1]:
from huggingface_hub import HfApi
import pandas as pd
from tqdm import tqdm
import ast

# Hugging Face API 인스턴스 생성
api = HfApi()

# 모델 리스트를 가져오기 (모델 메타 데이터 포함)
models_generator = api.list_models(full=True)

# 모델 메타 데이터를 저장할 리스트 초기화
models_data = []

# 모델 메타 데이터를 가져오는 동안 진행도 표시
for model in tqdm(models_generator, desc="Fetching model metadata"):
    models_data.append(model)

# 데이터프레임으로 변환
df = pd.DataFrame(models_data)

# 불필요한 컬럼 삭제
columns_to_drop = ['siblings', 'sha', 'mask_token', 'card_data', 'widget_data', 'model_index', 'config', 'transformers_info', 'spaces', 'safetensors']
df = df.drop(columns=columns_to_drop)

# 2. 새로운 컬럼 생성
df['arxiv'] = ''
df['dataset'] = ''
df['region'] = ''
df['license'] = ''

# 3. tags 컬럼의 객체들을 각 컬럼에 입력하고 tags에서 삭제
def process_tags(tags):
    arxiv, dataset, region, license = '', '', '', ''
    new_tags = []
    for tag in tags:
        lower_tag = tag.lower()
        if lower_tag.startswith('arxiv:'):
            arxiv = tag
        elif lower_tag.startswith('dataset:'):
            dataset = tag
        elif lower_tag.startswith('region:'):
            region = tag
        elif lower_tag.startswith('license:'):
            license = tag
        else:  # 특정 객체를 삭제하지 않음
            new_tags.append(tag)
    return arxiv, dataset, region, license, new_tags

# 진행도를 표시하면서 tags 컬럼 처리
for index in tqdm(range(len(df)), desc="Processing tags"):
    tags_data = df.at[index, 'tags']
    if isinstance(tags_data, str):
        tags_data = ast.literal_eval(tags_data)
    arxiv, dataset, region, license, new_tags = process_tags(tags_data)
    df.at[index, 'arxiv'] = arxiv
    df.at[index, 'dataset'] = dataset
    df.at[index, 'region'] = region
    df.at[index, 'license'] = license
    df.at[index, 'tags'] = new_tags

# 4. 수정된 데이터 프레임을 CSV 파일로 저장한다.
output_csv_file = '6-15.csv'
df.to_csv(output_csv_file, index=False)

Fetching model metadata: 717038it [06:48, 1756.69it/s]
Processing tags: 100%|███████████████████████████████████████████████████████| 717038/717038 [02:04<00:00, 5739.79it/s]


In [7]:
df

,id,author,created_at,last_modified,private,gated,disabled,downloads,likes,library_name,tags,pipeline_tag,arxiv,dataset,region,license
0,albert/albert-base-v1,albert,2022-03-02 23:29:04+00:00,2024-02-19 10:57:35+00:00,False,False,None,21912,7,transformers,"[transformers, pytorch, tf, safetensors, alber...",fill-mask,arxiv:1909.11942,dataset:wikipedia,region:us,license:apache-2.0
1,albert/albert-base-v2,albert,2022-03-02 23:29:04+00:00,2024-02-19 10:58:14+00:00,False,False,None,2509354,90,transformers,"[transformers, pytorch, tf, jax, rust, safeten...",fill-mask,arxiv:1909.11942,dataset:wikipedia,region:us,license:apache-2.0
2,albert/albert-large-v1,albert,2022-03-02 23:29:04+00:00,2024-02-19 10:58:26+00:00,False,False,None,2184,3,transformers,"[transformers, pytorch, tf, albert, fill-mask,...",fill-mask,arxiv:1909.11942,dataset:wikipedia,region:us,license:apache-2.0
3,albert/albert-large-v2,albert,2022-03-02 23:29:04+00:00,2024-02-19 10:58:48+00:00,False,False,None,68390,14,transformers,"[transformers, pytorch, tf, safetensors, alber...",fill-mask,arxiv:1909.11942,dataset:wikipedia,region:us,license:apache-2.0
4,albert/albert-xlarge-v1,albert,2022-03-02 23:29:04+00:00,2024-02-19 11:01:28+00:00,False,False,None,1597,4,transformers,"[transformers, pytorch, tf, safetensors, alber...",fill-mask,arxiv:1909.11942,dataset:wikipedia,region:us,license:apache-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665553,mazzaqq/SFT_Llama_Large,mazzaqq,2024-05-21 08:09:40+00:00,2024-05-21 08:09:41+00:00,False,False,None,0,0,None,[],None,,,region:us,
665554,viethq5/sft_llama3_v3,viethq5,2024-05-21 08:09:41+00:00,2024-05-21 08:09:43+00:00,False,False,None,0,0,None,[],None,,,region:us,
665555,KaranChand/phi-ft-100000-fp,KaranChand,2024-05-21 08:09:43+00:00,2024-05-21 08:09:43+00:00,False,False,None,0,0,None,[],None,,,region:us,
665556,AlignmentResearch/robust_llm_pythia-14m_pretra...,AlignmentResearch,2024-05-21 08:09:44+00:00,2024-05-21 08:09:44+00:00,False,False,None,0,0,None,[],None,,,region:us,


In [1]:
import pandas as pd
df = pd.read_csv("Model_2025-02-11.csv")

In [2]:
df.columns

Index(['id', 'author', 'sha', 'created_at', 'last_modified', 'private',
       'disabled', 'downloads', 'downloads_all_time', 'gated', 'gguf',
       'inference', 'likes', 'library_name', 'tags', 'pipeline_tag',
       'trending_score', 'security_repo_status', 'arxiv', 'dataset', 'region',
       'license'],
      dtype='object')

In [4]:
# library_name 컬럼에서 중복을 제거한 라이브러리 리스트 추출
unique_libraries = df['library_name'].dropna().unique()

# 라이브러리 이름 출력
print("Unique libraries in the dataset:")
for library in unique_libraries:
    print(library)

Unique libraries in the dataset:
transformers
diffusers
vllm
hunyuan3d-2.0
lerobot
ben2
birefnet
sentence-transformers
hibiki
coqui
transformers.js
liveportrait
trellis
chat_tts
pyannote-audio
gguf
granite-tsfm
f5-tts
diffusion-single-file
ultralytics
sam2
timesfm
mlx
ctranslate2
stable-audio-tools
hunyuan3d-1.0
cosyvoice
stable-diffusion
colpali
open-oasis
peft
moshi
t5
speechbrain
open_clip
seamless_communication
PyTorch
nemo
cosmos
fasttext
gliner
reverb
coreml
adapter-transformers
tf-keras
mlx-llm
pytorch
tflite
hunyuan-dit
timm
Tevatron
mini-omni2
model2vec
timer
speakerkit
bertopic
dghs-imgutils
sana
swarmformer
dghs-realutils
llamacpp
stable-baselines3
keras
ml-agents
espnet
fairseq
flair
pysentimiento
spacy
stanza
generic
tfhub
anime_segmentation
Doc-UFCN
txtai
yolov5
paddlenlp
TTS
gpt-neox
span-marker
tdc
tensorflowtts
allennlp
audiocraft
monai
fairseq2
wildlife-datasets
hezar
sklearn
setfit
clmbr
mlc-llm
RAGatouille
unity-sentis
Bunkatopics
trl
LLaVA
metavoice
llama.cpp
gemma

In [5]:
import pandas as pd
import ast

# CSV 파일 불러오기
file_path = 'Model_2025-02-11.csv'
df = pd.read_csv(file_path)

# 주어진 라이브러리 리스트들
libraries_list1 = [
    "transformers", "sentence-transformers", "keras", "adapter-transformers", "speechbrain",
    "espnet", "spacy", "fasttext", "asteroid", "generic", "ml-agents", "stable-baselines3",
    "pytorch", "flair", "allennlp", "fairseq", "k2", "tensorflowtts", "fastai", "opennmt",
    "timm", "sklearn", "paddlenlp", "superb", "doctr", "mindspore", "pytorch-ie", "pyannote-audio",
    "pysentimiento", "stanza", "PyTorch", "nemo", "txtai", "fastpitch", "pytorch-lightning",
    "PyTorch Lightning", "diffusers", "sample-factory", "k2-sherpa", "JoeyNMT", "FastAI",
    "stable-diffusion", "tfhub", "pythae", "open_clip", "mlconsole", "cleanrl", "setfit",
    "span-marker", "Doc-UFCN", "fastMONAI", "EveryDream", "kenlm", "BERT", "paddle_nlp",
    "rl-algo-impls", "PyLaia", "lucidrains/gated-state-spaces-pytorch", "TTS", "yolov5",
    "onnx", "yolov6detect", "ultralytics", "skrl", "yolor", "xpmir", "tensorflow", "peft",
    "scvi-tools", "hezar", "ppdiffusers", "mbrl-lib", "aster", "gpt-neox", "archai", "deep-rl-course",
    "Spacy", "ggml", "ctranslate2", "pcdet", "KerasCV Stable Diffusion in Diffusers", "tf",
    "tdc", "transformers.js", "PaddleNLP", "Transformers", "parking-env", "bertopic", "zeroshot_classifier",
    "ctransformers", "KerasNLP", "yolo", "safetensors", "safe-rlhf", "torch", "minetest-baselines",
    "audiocraft", "Core ML", "easydel", "wildlife-datasets", "monai", "fairseq2", "mlc", "Pytorch",
    "output-small", "braindecode", "coqui", "optimum_neuronx", "yasep", "bitsandbytes, transformers, peft, accelerate, bitsandbytes, datasets, deepspeed, trl",
    "htrflow_core", "xmen", "CTranslate2", "muzero-general", "pufferlib", "pytorch_geometric",
    "scikit-learn", "textgen", "peft - PEFT 0.5.0", "mlx", "DanishFungi", "asteroid1111111111111111111111111111111",
    "pruna-engine", "llama.cpp", "seamless_communication", "Spacy Explosion", "femr", "https://github.com/monet-joe/Piano-Classification",
    "clmbr", "xtuner", "fire-and-smoke", "TraceBERT", "Megatron-LM", "mlx-llm", "RAGatouille",
    "clinicadl", "unity-sentis", "ExLlamaV2", "tokenizers", "aim", "Bunkatopics",
    "transformers, Unsloth, Peft, trl, accelerate, bitsandbytes", "Flax", "ml-aim", "hierarchy-transformers",
    "Keras", "peft,sfttrainer", "whisperkit", "nanotron", "trl", "LLaVA", "transformers, peft, torch",
    "OpenVINO", "transformers, peft", "fasttext, bert", "tensorflow, keras", "metavoice", "gguf",
    "https://github.com/ICE-PIXIU", "colbert-ai", "gliner", "sglang", "non", "mlx-image",
    "GGUF", "gemma.cpp", "gemma_torch", "keras-nlp", "jax", "atommic", "mobileclip",
    "transformers(OpenNMT)", "v-jepa", "Transformers PHP", "whisper", "grok", "clot",
    "transformers, pe", "keras3", "UniFormer", "ml-4m", "fla", "UniFormerV2",
    "UniDepth", "voicecraft", "michelangelo", "openlm", "recurrentgemma", "tflite",
    "ditto", "PyTorchModelHubMixin-template", "Haystack", "elm", "litgpt, transformers",
    "dust3r", "rasa", "BiRefNet", "en-tts", "my-meta-test", "Gen AI", "zho-tts",
    "transformers accelerate bitsandbytes", "pyannote", "llama-cpp", "torchvision",
    "timesfm", "torchtune", "big_vision", "llamafile", "light-embed", "transformers, PyTorch",
    "edsnlp", "ultralyticsplus", "unsloth", "gptq", "chat_tts", "delphi", "burial_mounds",
    "fugent", "Nvidia Nemo", "segmentation-models-pytorch", "executorch", "multimolecule"
]

libraries_list2 = [
    "PyTorch", "TensorFlow", "JAX", "Transformers", "Safetensors", "TensorBoard", "PEFT",
    "Diffusers", "GGUF", "stable-baselines3", "ONNX", "ml-agents", "sentence-transformers",
    "Keras", "Adapters", "setfit", "timm", "sample-factory", "Flair", "Transformers.js",
    "MLX", "spaCy", "fastai", "ESPnet", "Core ML", "OpenVINO", "NeMo", "Joblib", "Rust",
    "BERTopic", "TF Lite", "fastText", "OpenCLIP", "Scikit-learn", "PaddlePaddle", "speechbrain",
    "Fairseq", "Graphcore", "Asteroid", "AllenNLP", "Stanza", "llamafile", "SpanMarker",
    "paddlenlp", "Habana", "pyannote.audio", "pythae", "unity-sentis"
]

# 첫 글자가 대문자인 경우와 소문자인 경우를 모두 포함한 라이브러리 리스트 생성
def create_case_insensitive_list(libraries):
    case_insensitive_list = set()
    for lib in libraries:
        case_insensitive_list.add(lib.lower())
        case_insensitive_list.add(lib.capitalize())
    return case_insensitive_list

normalized_libraries_list1 = set(lib.lower().replace('-', ' ') for lib in libraries_list1)
normalized_libraries_list2 = create_case_insensitive_list(libraries_list2)

# 새로운 컬럼 생성
df['library'] = ''

# tags 컬럼을 처리하여 새로운 library 컬럼에 값 넣기
for index, row in df.iterrows():
    tags = ast.literal_eval(row['tags'])
    libraries = [tag for tag in tags if tag.lower().replace('-', ' ') in normalized_libraries_list1 or tag.lower().replace('-', ' ') in normalized_libraries_list2]
    df.at[index, 'library'] = ', '.join(libraries)

# 결과를 CSV 파일로 저장
output_csv_file = 'Model_2025-02-11_output.csv'
df.to_csv(output_csv_file, index=False)

print(f"Processed {len(df)} entries and saved to {output_csv_file}.")

Processed 1407537 entries and saved to Model_2025-02-11_output.csv.


In [6]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# CSV 파일을 데이터프레임으로 불러오기
csv_file_path = 'Model_2025-02-11_output.csv'  # 실제 CSV 파일 경로로 변경하세요
df = pd.read_csv(csv_file_path)

# 각 라이브러리의 비율 계산
library_counts = df['library'].str.split(', ').explode().value_counts(normalize=True)

# 정규화 함수
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

# 로그 변환 및 정규화된 좋아요 수와 다운로드 수 계산
df['log_likes'] = np.log1p(df['likes'])  # 로그 변환 적용
df['normalized_log_likes'] = normalize(df['log_likes'])
df['log_downloads'] = np.log1p(df['downloads'])  # 로그 변환 적용
df['normalized_log_downloads'] = normalize(df['log_downloads'])

# 업데이트 주기 계산 함수
def calculate_update_frequency(created_at, last_modified):
    if pd.isna(created_at) or pd.isna(last_modified):
        return 0
    created_date = datetime.strptime(created_at, "%Y-%m-%d %H:%M:%S%z")
    last_modified_date = datetime.strptime(last_modified, "%Y-%m-%d %H:%M:%S%z")
    days_since_creation = (last_modified_date - created_date).days
    return days_since_creation

# 업데이트 주기 계산 및 1년 내 업데이트 여부 체크
df['update_frequency'] = df.apply(lambda row: calculate_update_frequency(row['created_at'], row['last_modified']), axis=1)
df['last_modified_date'] = pd.to_datetime(df['last_modified'], format="%Y-%m-%d %H:%M:%S%z")
one_year_ago = datetime.now(datetime.now().astimezone().tzinfo) - timedelta(days=365)
df['within_one_year'] = df['last_modified_date'] > one_year_ago

# 1년 내 업데이트된 데이터만을 사용하여 정규화 진행
df_within_one_year = df[df['within_one_year']].copy()
df_within_one_year['normalized_update_frequency'] = normalize(df_within_one_year['update_frequency'])

# 정규화된 업데이트 주기 병합
df = df.merge(df_within_one_year[['id', 'normalized_update_frequency']], on='id', how='left')
df['normalized_update_frequency'] = df['normalized_update_frequency'].fillna(0)

# 기준 점수 설정
BASE_LOG_LIKES_SCORE = 30
BASE_UPDATE_FREQUENCY_SCORE = 10
BASE_LOG_DOWNLOADS_SCORE = 50
BASE_LIBRARY_SCORE = 10
AUTO_GATED_PENALTY = -10
MANUAL_GATED_PENALTY = -20

# 영향력 점수 계산 함수
def calculate_influence_score(row):
    scores = {}
    
    # 좋아요 수 기반 점수 계산 (정규화된 로그 좋아요 수 * 30점)
    log_likes_score = row['normalized_log_likes'] * BASE_LOG_LIKES_SCORE
    scores['log_likes_score'] = log_likes_score
    
    # 업데이트 주기 기반 점수 계산 (정규화된 업데이트 주기 * 10점)
    update_frequency_score = row['normalized_update_frequency'] * BASE_UPDATE_FREQUENCY_SCORE
    scores['update_frequency_score'] = update_frequency_score
    
    # 최근 30일 간 다운로드 수 기반 점수 계산 (정규화된 로그 다운로드 수 * 50점)
    log_downloads_score = row['normalized_log_downloads'] * BASE_LOG_DOWNLOADS_SCORE
    scores['log_downloads_score'] = log_downloads_score
    
    # 라이브러리 이름 기반 점수 계산 (비율 기반 가중치 적용, 최대 10점)
    library_names = row['library_name'].split(', ') if pd.notna(row['library_name']) else []
    library_weight = sum(library_counts.get(lib, 0) * BASE_LIBRARY_SCORE for lib in library_names)
    scores['library_weight'] = library_weight

    # 게이트 처리 여부 점수 조정
    if row['gated'] == 'auto':
        gated_penalty = AUTO_GATED_PENALTY
    elif row['gated'] == 'manual':
        gated_penalty = MANUAL_GATED_PENALTY
    else:
        gated_penalty = 0
    scores['gated_penalty'] = gated_penalty
    
    # 총합 점수 계산
    total_score = log_likes_score + update_frequency_score + log_downloads_score + scores['library_weight'] + gated_penalty
    scores['total_score'] = total_score
    
    return pd.Series(scores)

# 새로운 데이터프레임 생성
scores_df = df.apply(calculate_influence_score, axis=1)
result_df = df[['id', 'likes', 'created_at', 'last_modified', 'downloads', 'library_name', 'gated']].join(scores_df)

# 인덱스 초기화
result_df = result_df.reset_index(drop=True)

# CSV 파일로 저장
result_df.to_csv('Model_2025-02-11_score.csv', index=False)

In [7]:
import pandas as pd

# CSV 파일 열기
file_path = "Model_2025-02-11_output.csv"  # 실제 파일 경로로 변경하세요
data = pd.read_csv(file_path)

# 총 다운로드 수와 좋아요 수 계산
total_downloads = data['downloads'].sum()
total_likes = data['likes'].sum()

# 다운로드 수와 좋아요 수의 비율 계산 (총 다운로드 수 / 총 좋아요 수)
ratio = total_downloads / total_likes  # 예: 1980.65

# 다운로드 점수를 5/3로 조정
data['download_score'] = (data['downloads'] / ratio) * (5 / 3)
data['like_score'] = data['likes'] * 1

# 최종 점수 계산: 다운로드 수 점수 + 좋아요 수 점수
data['total_score'] = data['download_score'] + data['like_score']

# 정렬된 데이터 출력 (정렬 없이 저장)
data.to_csv("Model_2025-02-11_influence.csv", index=False)

In [8]:
import pandas as pd

# 각 CSV 파일의 경로
file_paths = [
    'Model_2025-02-11_output.csv', 
    'Model_2025-02-11_score.csv', 
    'Model_2025-02-11_influence.csv'
]

# 각 파일에서 필요한 컬럼 이름과 새로운 이름 (일부만 변경)
columns_to_extract_and_rename = {
    'Model_2025-02-11_output.csv': {
        'id': 'id',
        'author': 'author',
        'created_at': 'created_at',
        'last_modified': 'last_modified',
        'private': 'private',
        'gated': 'gated',
        'disabled': 'disabled',
        'downloads': 'downloads',
        'likes': 'likes',
        'library_name': 'library_name',
        'library': 'all_library',
        'tags': 'tags',
        'pipeline_tag': 'pipeline_tag',
        'arxiv': 'arxiv',
        'dataset': 'dataset',
        'license': 'license'
    },
    'Model_2025-02-11_score.csv': {
        'total_score': 'model_score'
    },
    'Model_2025-02-11_influence.csv': {
        'total_score': 'influence_score'
    }
}

# 빈 데이터프레임 생성
combined_df = pd.DataFrame()

# 각 파일을 열고 필요한 컬럼만 추출 및 이름 변경하여 합치기
for file_path in file_paths:
    df = pd.read_csv(file_path)
    columns_to_rename = columns_to_extract_and_rename[file_path]
    df_selected = df[list(columns_to_rename.keys())]
    df_selected.rename(columns=columns_to_rename, inplace=True)
    combined_df = pd.concat([combined_df, df_selected], axis=1)

# 새로운 CSV 파일로 저장
combined_df.to_csv('Model_2025-02-11_final.csv', index=False)

print("새로운 CSV 파일로 저장되었습니다: 6-11_final.csv")

C:\Users\kodonghwan\AppData\Local\Temp\ipykernel_16808\2404542778.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.rename(columns=columns_to_rename, inplace=True)
C:\Users\kodonghwan\AppData\Local\Temp\ipykernel_16808\2404542778.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.rename(columns=columns_to_rename, inplace=True)
C:\Users\kodonghwan\AppData\Local\Temp\ipykernel_16808\2404542778.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

새로운 CSV 파일로 저장되었습니다: 6-11_final.csv


In [9]:
import pandas as pd

# CSV 파일 불러오기
file_path = 'Model_2025-02-11_final.csv'  # 파일 경로를 업데이트하세요
df = pd.read_csv(file_path)

# 'influence_score' 컬럼을 기준으로 데이터프레임 정렬
df_sorted = df.sort_values(by='influence_score', ascending=False)

# 상위 10000개의 행 추출
df_top_10000 = df_sorted.head(10000)

# 추출된 데이터프레임을 인덱스 순서대로 다시 정렬
df_top_10000_sorted_by_index = df_top_10000.sort_index()

# 결과 데이터프레임 출력
print(df_top_10000_sorted_by_index.head())

# 결과 데이터프레임을 새로운 CSV 파일로 저장 (선택사항)
df_top_10000_sorted_by_index.to_csv('Model_2025-02-11_influence_top_10000.csv', index=False)

                                         id       author  \
0                   deepseek-ai/DeepSeek-R1  deepseek-ai   
1  ValueFX9507/Tifa-Deepsex-14b-CoT-GGUF-Q4  ValueFX9507   
2                  deepseek-ai/Janus-Pro-7B  deepseek-ai   
3                  Zyphra/Zonos-v0.1-hybrid       Zyphra   
4                        hexgrad/Kokoro-82M      hexgrad   

                  created_at              last_modified  private  gated  \
0  2025-01-20 03:46:07+00:00  2025-02-09 06:34:11+00:00    False  False   
1  2025-02-04 07:20:55+00:00  2025-02-08 09:58:51+00:00    False  False   
2  2025-01-26 12:05:50+00:00  2025-02-01 08:00:16+00:00    False  False   
3  2025-02-06 22:35:07+00:00  2025-02-10 18:44:11+00:00    False  False   
4  2024-12-26 00:20:08+00:00  2025-02-01 23:54:42+00:00    False  False   

   disabled  downloads  likes  library_name                all_library  \
0       NaN    2670920   8176  transformers  transformers, safetensors   
1       NaN      40272    409  transform

In [4]:
import os
import pandas as pd
from tqdm import tqdm

# 1. top_10000_influence_score.csv을 열어 df에 저장
df = pd.read_csv('6-11_influence_top_10000.csv')

# 2. df의 id 컬럼의 데이터를 가져옴
ids = df['id']

# 3. df_downloads, df_likes 데이터프레임 생성
df_downloads = pd.DataFrame(ids)
df_likes = pd.DataFrame(ids)

# 4. C:\Users\han\모델 메타데이터에 있는 모든 csv 파일을 순차적으로 염
directory = 'C:\\Users\\han\\모델 메타데이터'
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

# 반복 작업
for file in tqdm(csv_files, desc='Processing files'):
    file_path = os.path.join(directory, file)
    temp_df = pd.read_csv(file_path)

    # 5. 파일의 이름을 제목으로 df_downloads, df_likes에 새로운 컬럼으로 추가
    column_name = os.path.splitext(file)[0]
    df_downloads[column_name] = 0
    df_likes[column_name] = 0

    # 6. df의 id 컬럼과 순차적으로 연 파일의 id 컬럼을 비교하여 존재하는 id의 행만 추출
    merged_df = pd.merge(df[['id']], temp_df[['id', 'downloads', 'likes']], on='id', how='inner')

    # 7. 추출한 행의 downloads와 likes를 각 파일 이름과 동일한 컬럼에 입력
    df_downloads.loc[df_downloads['id'].isin(merged_df['id']), column_name] = merged_df['downloads'].values
    df_likes.loc[df_likes['id'].isin(merged_df['id']), column_name] = merged_df['likes'].values

# 8. df_downloads와 df_likes를 각각 csv 파일로 저장
df_downloads.to_csv('df_downloads.csv', index=False)
df_likes.to_csv('df_likes.csv', index=False)

Processing files: 100%|████████████████████████████████████████████████████████████████| 25/25 [01:23<00:00,  3.35s/it]


In [5]:
import pandas as pd

# CSV 파일 로드
file_path = 'df_likes.csv'  # 실제 파일 경로로 변경해주세요
df = pd.read_csv(file_path)

# 첫 번째 컬럼(id)를 제외한 모든 컬럼 선택
df_likes = df.iloc[:, 1:]

# 좋아요 수 증감량 계산
likes_diff = df_likes.diff(axis=1)

# 좋아요 수 증감률 계산
likes_diff_percent = df_likes.pct_change(axis=1) * 100

# 결과를 하나의 DataFrame으로 결합
result_df = pd.concat([df['id'], likes_diff, likes_diff_percent], axis=1)
result_df.columns = ['id'] + [f'{col}_diff' for col in df_likes.columns] + [f'{col}_diff_percent' for col in df_likes.columns]

# 결과를 CSV 파일로 저장
output_file_path = 'likes_changes.csv'  # 실제 파일 경로로 변경해주세요
result_df.to_csv(output_file_path, index=False)

print(f'CSV 파일이 {output_file_path}에 저장되었습니다.')

CSV 파일이 likes_changes.csv에 저장되었습니다.


In [2]:
from huggingface_hub import HfApi
import pandas as pd
from tqdm import tqdm

# Hugging Face API 인스턴스 생성
api = HfApi()

# 데이터셋 리스트를 가져오기 (데이터셋 메타 데이터 포함)
datasets_generator = api.list_datasets(full=True)

# 데이터셋 메타 데이터를 저장할 리스트 초기화
datasets_data = []

# 데이터셋 메타 데이터를 가져오는 동안 진행도 표시
for dataset in tqdm(datasets_generator, desc="Fetching dataset metadata"):
    datasets_data.append(dataset)

# 데이터프레임으로 변환
df = pd.DataFrame(datasets_data)

# 데이터프레임을 CSV 파일로 저장 (선택 사항)
df.to_csv('datasets_metadata.csv', index=False)

print("Data fetched and saved to CSV file successfully!")

Fetching dataset metadata: 163262it [02:07, 1282.54it/s]


Data fetched and saved to CSV file successfully!
